# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 11 2022 11:06AM,243956,15,8384527,"Carwin Pharmaceutical Associates, LLC",Released
1,Aug 11 2022 11:06AM,243956,15,8384541,"Carwin Pharmaceutical Associates, LLC",Released
2,Aug 11 2022 11:06AM,243956,15,8384542,"Carwin Pharmaceutical Associates, LLC",Released
3,Aug 11 2022 11:06AM,243956,15,8385720,"Carwin Pharmaceutical Associates, LLC",Released
4,Aug 11 2022 10:46AM,243955,15,ALI426105-0578,"Alliance Pharma, Inc.",Released
5,Aug 11 2022 10:46AM,243955,15,30009885,"Alliance Pharma, Inc.",Released
6,Aug 11 2022 10:46AM,243955,15,30009886,"Alliance Pharma, Inc.",Released
7,Aug 11 2022 10:46AM,243955,15,30009889,"Alliance Pharma, Inc.",Released
8,Aug 11 2022 10:46AM,243955,15,30009891,"Alliance Pharma, Inc.",Released
9,Aug 11 2022 10:46AM,243955,15,30009890,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,243952,Released,1
37,243953,Released,1
38,243954,Released,14
39,243955,Released,40
40,243956,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243952,NaN,NaN,1.0
243953,NaN,NaN,1.0
243954,NaN,NaN,14.0
243955,NaN,NaN,40.0
243956,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243871,6.0,6.0,2.0
243897,0.0,1.0,0.0
243900,0.0,0.0,1.0
243901,0.0,0.0,1.0
243905,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243871,6,6,2
243897,0,1,0
243900,0,0,1
243901,0,0,1
243905,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243871,6,6,2
1,243897,0,1,0
2,243900,0,0,1
3,243901,0,0,1
4,243905,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243871,6,6,2
1,243897,,1,
2,243900,,,1
3,243901,,,1
4,243905,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 11 2022 11:06AM,243956,15,"Carwin Pharmaceutical Associates, LLC"
4,Aug 11 2022 10:46AM,243955,15,"Alliance Pharma, Inc."
44,Aug 11 2022 10:42AM,243954,12,Hush Hush
58,Aug 11 2022 10:36AM,243953,19,"GUSA Granules USA, Inc."
59,Aug 11 2022 10:31AM,243952,12,Hush Hush
60,Aug 11 2022 10:28AM,243951,10,Beach Products Inc
61,Aug 11 2022 10:24AM,243950,10,Eywa Pharma Inc.
65,Aug 11 2022 10:23AM,243949,10,ISDIN Corporation
86,Aug 11 2022 10:23AM,243946,10,Eywa Pharma Inc.
115,Aug 11 2022 10:14AM,243948,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 11 2022 11:06AM,243956,15,"Carwin Pharmaceutical Associates, LLC",,,4
1,Aug 11 2022 10:46AM,243955,15,"Alliance Pharma, Inc.",,,40
2,Aug 11 2022 10:42AM,243954,12,Hush Hush,,,14
3,Aug 11 2022 10:36AM,243953,19,"GUSA Granules USA, Inc.",,,1
4,Aug 11 2022 10:31AM,243952,12,Hush Hush,,,1
5,Aug 11 2022 10:28AM,243951,10,Beach Products Inc,,,1
6,Aug 11 2022 10:24AM,243950,10,Eywa Pharma Inc.,,,4
7,Aug 11 2022 10:23AM,243949,10,ISDIN Corporation,,,21
8,Aug 11 2022 10:23AM,243946,10,Eywa Pharma Inc.,,,29
9,Aug 11 2022 10:14AM,243948,10,"Methapharm, Inc.",,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 11 2022 11:06AM,243956,15,"Carwin Pharmaceutical Associates, LLC",4,,
1,Aug 11 2022 10:46AM,243955,15,"Alliance Pharma, Inc.",40,,
2,Aug 11 2022 10:42AM,243954,12,Hush Hush,14,,
3,Aug 11 2022 10:36AM,243953,19,"GUSA Granules USA, Inc.",1,,
4,Aug 11 2022 10:31AM,243952,12,Hush Hush,1,,
5,Aug 11 2022 10:28AM,243951,10,Beach Products Inc,1,,
6,Aug 11 2022 10:24AM,243950,10,Eywa Pharma Inc.,4,,
7,Aug 11 2022 10:23AM,243949,10,ISDIN Corporation,21,,
8,Aug 11 2022 10:23AM,243946,10,Eywa Pharma Inc.,29,,
9,Aug 11 2022 10:14AM,243948,10,"Methapharm, Inc.",6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 11 2022 11:06AM,243956,15,"Carwin Pharmaceutical Associates, LLC",4,,
1,Aug 11 2022 10:46AM,243955,15,"Alliance Pharma, Inc.",40,,
2,Aug 11 2022 10:42AM,243954,12,Hush Hush,14,,
3,Aug 11 2022 10:36AM,243953,19,"GUSA Granules USA, Inc.",1,,
4,Aug 11 2022 10:31AM,243952,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 11 2022 11:06AM,243956,15,"Carwin Pharmaceutical Associates, LLC",4.0,NaN,NaN
1,Aug 11 2022 10:46AM,243955,15,"Alliance Pharma, Inc.",40.0,NaN,NaN
2,Aug 11 2022 10:42AM,243954,12,Hush Hush,14.0,NaN,NaN
3,Aug 11 2022 10:36AM,243953,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
4,Aug 11 2022 10:31AM,243952,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 11 2022 11:06AM,243956,15,"Carwin Pharmaceutical Associates, LLC",4.0,0.0,0.0
1,Aug 11 2022 10:46AM,243955,15,"Alliance Pharma, Inc.",40.0,0.0,0.0
2,Aug 11 2022 10:42AM,243954,12,Hush Hush,14.0,0.0,0.0
3,Aug 11 2022 10:36AM,243953,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
4,Aug 11 2022 10:31AM,243952,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4390916,395.0,8.0,0.0
12,487906,15.0,0.0,0.0
15,1219647,53.0,0.0,0.0
16,243909,1.0,0.0,0.0
17,731739,15.0,0.0,0.0
19,731782,13.0,13.0,9.0
20,487811,31.0,12.0,6.0
21,975605,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4390916,395.0,8.0,0.0
1,12,487906,15.0,0.0,0.0
2,15,1219647,53.0,0.0,0.0
3,16,243909,1.0,0.0,0.0
4,17,731739,15.0,0.0,0.0
5,19,731782,13.0,13.0,9.0
6,20,487811,31.0,12.0,6.0
7,21,975605,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,395.0,8.0,0.0
1,12,15.0,0.0,0.0
2,15,53.0,0.0,0.0
3,16,1.0,0.0,0.0
4,17,15.0,0.0,0.0
5,19,13.0,13.0,9.0
6,20,31.0,12.0,6.0
7,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,395.0
1,12,Released,15.0
2,15,Released,53.0
3,16,Released,1.0
4,17,Released,15.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0
Executing,8.0,0.0,0.0,0.0,0.0,13.0,12.0,1.0
Released,395.0,15.0,53.0,1.0,15.0,13.0,31.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0
1,Executing,8.0,0.0,0.0,0.0,0.0,13.0,12.0,1.0
2,Released,395.0,15.0,53.0,1.0,15.0,13.0,31.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0
1,Executing,8.0,0.0,0.0,0.0,0.0,13.0,12.0,1.0
2,Released,395.0,15.0,53.0,1.0,15.0,13.0,31.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()